In [1]:
import os, re, datetime
import pandas as pd
from collections import OrderedDict
from docxtpl import DocxTemplate

In [2]:
df = pd.read_excel('乳腺癌化疗用药数据库0822.xlsx')

In [3]:
df = df.dropna(axis = 0, subset = ['类别'] )

In [4]:
#填充nan值的函数
def make_column(string):
    a = ''
    list1 = []
    for i in df.index:
        if type(df[string][i]) != type(df[string][2]):
            a = df[string][i]
            list1.append(a)
        else:
            list1.append(a)
    return list1

for i in ['化疗药物', '基因', '位点', '证据等级']:
    df[i] = (make_column(i))

In [5]:
dic = {}
dic1 = {}
for i in df.index:
    dic.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})
    dic1.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})

    dic[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['解释'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['类别'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]]['证据等级'] = df['证据等级'][i]

In [6]:
list1 = ['rs8133052', 'rs1801133', 'rs67376798', 'rs1042858', 'rs11598702', 'rs7984870']
list2 = ['rs11615']

list3 = []

In [7]:
def modify_result(df_name, list1, list2, list3):
    #list1:基因型与数据库相反的位点列表，list2:基因型与数据库反向互补的位点列表 
    #list3:基因型与数据库互补的位点列表
    reverse_complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()[::-1]
    complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()
    for i in range(df_name.shape[0]):
        for line1 in list1:
            df_name.at[i, line1] = df_client.at[i, line1][::-1]#将检测结果中与数据库相反的基因型校正
        for line2 in list2:
            df_name.at[i, line2] = reverse_complement(df_name.at[i, line2])
        for line3 in list3:
            df_name.at[i, line3] = complement(df_name.at[i, line3])

In [8]:
df_client = pd.read_excel('乳腺癌检测报告.xlsx')
modify_result(df_client, list1, list2, list3)
list_rs = re.findall(r'rs\d+', str(df_client.columns))
dic_client = {}
for i in range(df_client.shape[0]):
    for j in list_rs:
        dic_client.setdefault (df_client['姓名'][i], {})
        dic_client[df_client['姓名'][i]][j] = df_client[j][i]

In [9]:
#输入客户名字，产生临床提示结果
def clinical_prompt(client_name ):
    d_result = {}#临床提示映射
    list_pinggu = []
    d_test = {}
    for i in dic1.keys():
        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] )
        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))
    for i in dic_pinggu_result.keys():
        d_test.setdefault(i.split('\t')[0],[])
        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])
    #for i, a in d_test.items():
    for i, a in d_test.items():
        if len(a) == 1:
            d_result[i] = a[0]
        if len(a) > 1:
            if i == '来曲唑' or i == '阿那曲唑':
                d_result[i] = '，'.join(a)
            else:
                
                list_effect = []
                
                dic_effect = {'疗效':('较差', '适中', '较好'), '毒副':('较低', '适中', '较高'), '代谢':('较慢', '适中', '较快'),}
                du = re.findall(r'毒副\w+', str(a))
                dai = re.findall(r'代谢\w+', str(a))
                liao = re.findall(r'疗效\w+', str(a))
                for j in [liao, du, dai]: #临床提示顺序
                    if len(j) > 1:
                        for line in dic_effect.keys():
                            if j[0][:2] == line:
                
                                if len(re.findall(re.escape(dic_effect[line][0]), str(a))) and len(re.findall(re.escape(dic_effect[line][2]), str(a))) or len(re.findall(re.escape(dic_effect[line][0]), str(a))) and len(re.findall(re.escape(dic_effect[line][1]), str(a))):
                                    list_effect.append(line + dic_effect[line][1])
                                if len(re.findall(re.escape(dic_effect[line][1]), str(a))) and len(re.findall(re.escape(dic_effect[line][2]), str(a))) and len(re.findall(re.escape(dic_effect[line][0]), str(a))) == 0:
                                    list_effect.append(line + dic_effect[line][2])
                                if len(re.findall(re.escape(dic_effect[line][0]), str(a))) and len(re.findall(re.escape(dic_effect[line][2]), str(a))) == 0 and len(re.findall(re.escape(dic_effect[line][1]), str(a))) == 0:
                                    list_effect.append(line + dic_effect[line][0])
                                if len(re.findall(re.escape(dic_effect[line][0]), str(a))) and len(re.findall(re.escape(dic_effect[line][2]), str(a))) and len(re.findall(re.escape(dic_effect[line][1]), str(a))):
                                    list_effect.append(line + dic_effect[line][1])
                                if len(re.findall(re.escape(dic_effect[line][2]), str(a))) and len(re.findall(re.escape(dic_effect[line][1]), str(a))) == 0 and len(re.findall(re.escape(dic_effect[line][0]), str(a))) == 0:
                                    list_effect.append(line + dic_effect[line][2])
                    if len(j) == 1:
                        list_effect.append(j[0])
                    else:
                        pass
                d_result[i] = '，'.join(list_effect)
    return d_result
            


In [10]:
def gene_type_level(name):
    gene_type_dic = {}
    for i in dic1.keys():
        gene_type_dic.setdefault(i, {})
        gene_type_dic[i]['基因型'] = dic_client[name][i.split('\t')[-1]]
        gene_type_dic[i]['证据等级'] = dic1[i]['证据等级']
        text = dic1[i][dic_client[name][i.split('\t')[2]]]
        dai = re.findall(r'代谢', text)
        du = re.findall(r'毒副', text)
        liao = list(set(re.findall(r'疗效', text)))
        x = []
        for k in [liao, du, dai]:
            if k:
                x += k
        #print('，'.join(x).replace('副', '性'))
        
        

        gene_type_dic[i]['类别'] = '，'.join(x).replace('副', '性')
            
        
    return gene_type_dic

In [31]:
dic_info_client = {}
for i in range(df_client.shape[0]):
    dic_info_client.setdefault(df_client['姓名'][i], {})
    dic_info_client[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    dic_info_client[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    dic_info_client[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    dic_info_client[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dic_info_client[df_client['姓名'][i]]['临床提示'] = clinical_prompt(df_client['姓名'][i])
    dic_info_client[df_client['姓名'][i]]['基因型'] = gene_type_level(df_client['姓名'][i])
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str(dt.month), str(dt.day))
    dic_info_client[df_client['姓名'][i]]['报告日期'] = reportdate

In [32]:
dic_info_client['宋士云']

{'采样日期': '2019-08-08',
 '收样日期': '2019-08-11',
 '姓名': '宋士云',
 '性别': '女',
 '年龄': 67,
 '样本编号': '17B010563',
 '临床提示': {'环磷酰胺+表柔比星': '疗效较好，毒副较低',
  '环磷酰胺+多柔比星+氟尿嘧啶': '毒副较高',
  '多柔比星': '疗效较差，毒副较低',
  '环磷酰胺': '疗效适中，毒副适中',
  '紫杉醇': '疗效较差，毒副较低',
  '多西他赛': '疗效较差，毒副较高',
  '顺铂': '疗效较好，毒副适中',
  '卡铂': '疗效较好，毒副适中',
  '卡培他滨': '疗效较好，毒副适中，代谢较快',
  '氟尿嘧啶': '疗效较好，毒副适中，代谢较快',
  '吉西他滨': '疗效较好，毒副较低，代谢较慢',
  '甲氨蝶呤': '疗效较差，毒副较高',
  '他莫昔芬': '疗效较好，毒副较高',
  '来曲唑': '绝经前疗效较差，绝经后疗效较好，毒副较低',
  '阿那曲唑': '绝经前疗效较差，绝经后疗效较好，毒副较低',
  '依西美坦': '毒副适中'},
 '基因型': {'环磷酰胺+表柔比星\tGSTP1\trs1695': {'基因型': 'AG',
   '证据等级': '2A',
   '类别': '疗效，毒性'},
  '环磷酰胺+多柔比星+氟尿嘧啶\tABCC4\trs9561778': {'基因型': 'GT', '证据等级': 3, '类别': '毒性'},
  '环磷酰胺+多柔比星+氟尿嘧啶\tERCC1\trs11615': {'基因型': 'GG', '证据等级': 3, '类别': '毒性'},
  '多柔比星\tCBR3\trs8133052': {'基因型': 'GG', '证据等级': 3, '类别': '疗效，毒性'},
  '环磷酰胺\tSOD2\trs4880': {'基因型': 'AG', '证据等级': '2B', '类别': '疗效'},
  '环磷酰胺\tABCC4\trs9561778': {'基因型': 'GT', '证据等级': 3, '类别': '毒性'},
  '紫杉醇\tABCB1\trs1045642': {'基因型': 'AA', '证据等级

In [55]:
def jr(name):
    result_list = []
    for drug in dic_info_client[name]['临床提示'].keys():
        judge_list = []
        result_dic = {}
        result_dic['Drugs'] = drug
        result_dic['Estimate'] = dic_info_client[name]['临床提示'][drug]
        result_dic['detail'] = []
        for sub_key, sub_value in dic_info_client[name]['基因型'].items():
            if drug == sub_key.split('\t')[0]:
                #detail_dic = {}
                if sub_key.split('\t')[1] not in judge_list:
                    judge_list.append(sub_key.split('\t')[1])
                    detail_dic = {}
                    detail_dic['Gene'] = sub_key.split('\t')[1]
                    detail_dic['new_detail'] = []
                    newdetail_dic = {}
                    newdetail_dic['rsid'] = sub_key.split('\t')[2]
                    newdetail_dic['GT'] = sub_value['基因型']
                    newdetail_dic['type'] = sub_value['类别']
                    newdetail_dic['level'] = sub_value['证据等级']
                    detail_dic['new_detail'].append(newdetail_dic)
                    result_dic['detail'].append(detail_dic)
                else:
                    newdetail_dic = {}
                    newdetail_dic['rsid'] = sub_key.split('\t')[2]
                    newdetail_dic['GT'] = sub_value['基因型']
                    newdetail_dic['type'] = sub_value['类别']
                    newdetail_dic['level'] = sub_value['证据等级']
                    for i in result_dic['detail']:
                        if i['Gene'] == sub_key.split('\t')[1]:
                            i['new_detail'].append(newdetail_dic)
                    #result_dic['detail'].append(detail_dic)
        result_list.append(result_dic)
    return result_list

In [62]:
jr('宋士云')

[{'Drugs': '环磷酰胺+表柔比星',
  'Estimate': '疗效较好，毒副较低',
  'detail': [{'Gene': 'GSTP1',
    'new_detail': [{'rsid': 'rs1695',
      'GT': 'AG',
      'type': '疗效，毒性',
      'level': '2A'}]}]},
 {'Drugs': '环磷酰胺+多柔比星+氟尿嘧啶',
  'Estimate': '毒副较高',
  'detail': [{'Gene': 'ABCC4',
    'new_detail': [{'rsid': 'rs9561778',
      'GT': 'GT',
      'type': '毒性',
      'level': 3}]},
   {'Gene': 'ERCC1',
    'new_detail': [{'rsid': 'rs11615',
      'GT': 'GG',
      'type': '毒性',
      'level': 3}]}]},
 {'Drugs': '多柔比星',
  'Estimate': '疗效较差，毒副较低',
  'detail': [{'Gene': 'CBR3',
    'new_detail': [{'rsid': 'rs8133052',
      'GT': 'GG',
      'type': '疗效，毒性',
      'level': 3}]}]},
 {'Drugs': '环磷酰胺',
  'Estimate': '疗效适中，毒副适中',
  'detail': [{'Gene': 'SOD2',
    'new_detail': [{'rsid': 'rs4880',
      'GT': 'AG',
      'type': '疗效',
      'level': '2B'}]},
   {'Gene': 'ABCC4',
    'new_detail': [{'rsid': 'rs9561778',
      'GT': 'GT',
      'type': '毒性',
      'level': 3}]}]},
 {'Drugs': '紫杉醇',
  'Estimate'

In [80]:
finall_dic = {}
for i in range(df_client.shape[0]):
    finall_dic.setdefault(df_client['姓名'][i], {})
    finall_dic[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    finall_dic[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    finall_dic[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    finall_dic[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str(dt.month), str(dt.day))
    finall_dic[df_client['姓名'][i]]['报告日期'] = reportdate
    finall_dic[df_client['姓名'][i]]['Result'] =  jr(df_client['姓名'][i])

In [81]:
finall_dic

{'蔡月桂': {'采样日期': '2019-08-07',
  '收样日期': '2019-08-09',
  '姓名': '蔡月桂',
  '性别': '女',
  '年龄': 61,
  '样本编号': '17B010560',
  '报告日期': '2019-8-28',
  'Result': [{'Drugs': '环磷酰胺+表柔比星',
    'Estimate': '疗效较好，毒副较低',
    'detail': [{'Gene': 'GSTP1',
      'new_detail': [{'rsid': 'rs1695',
        'GT': 'AG',
        'type': '疗效，毒性',
        'level': '2A'}]}]},
   {'Drugs': '环磷酰胺+多柔比星+氟尿嘧啶',
    'Estimate': '毒副适中',
    'detail': [{'Gene': 'ABCC4',
      'new_detail': [{'rsid': 'rs9561778',
        'GT': 'GG',
        'type': '毒性',
        'level': 3}]},
     {'Gene': 'ERCC1',
      'new_detail': [{'rsid': 'rs11615',
        'GT': 'GG',
        'type': '毒性',
        'level': 3}]}]},
   {'Drugs': '多柔比星',
    'Estimate': '疗效适中，毒副适中',
    'detail': [{'Gene': 'CBR3',
      'new_detail': [{'rsid': 'rs8133052',
        'GT': 'AG',
        'type': '疗效，毒性',
        'level': 3}]}]},
   {'Drugs': '环磷酰胺',
    'Estimate': '疗效适中，毒副较低',
    'detail': [{'Gene': 'SOD2',
      'new_detail': [{'rsid': 'rs4880',
    

In [83]:
file = 'template.docx'
#tpl = DocxTemplate(file)
for name in finall_dic.keys():
    outdoc = re.sub('template',name , file)
    tpl = DocxTemplate(file)
    tpl.render(finall_dic[name])
    tpl.save(outdoc)
    #finall_dic[name]